In [330]:
#数据挖掘0313作业
#统计17 张韵文 2017310819

import numpy as np
from scipy import linalg
import os
import sys

def solve_sym(xwx, s):
    L = linalg.cholesky(xwx)
    return linalg.lapack.dpotrs(L, s)[0]

def turnbits_rec(p):
    if(p==1):
        return np.array([[True, False],[True, True]])
    else:
        tmp1 = np.c_[ turnbits_rec(p-1), np.array([False]*(2**(p-1))).reshape((2**(p-1),1))]
        tmp2 = np.c_[ turnbits_rec(p-1), np.array([True]*(2**(p-1))).reshape((2**(p-1),1))]
        return np.r_[tmp1, tmp2]

class BestSubsetReg(object):
    def __init__(self, x=0, y=0, inter=True, isCp=True, isAIC=True, isCV=True):
        self.n, self.p = x.shape
        if inter:
            self.x = np.c_[np.ones((self.n,1)),x]
        else:
            self.x = x
        self.y = y
        self.xtx = np.dot(self.x.T,self.x)
        self.xty = np.dot(self.x.T,self.y)
        self.ind_var = turnbits_rec(self.p)
        self.b = [solve_sym(self.xtx[ind][:,ind], self.xty[ind]) for ind in self.ind_var]
        self.isCp = isCp
        self.isAIC = isAIC
        self.isCV = isCV
        if self.isCp:
            self.Cp = 0
        if self.isAIC:
            self.AIC = 0
        if self.isCV:
            self.cverr = 0
            
    def Cp_AIC(self):
        mse_tmp = [np.sum(np.dot(self.xtx[ind][:,ind],b_)*b_)for ind, b_ in zip(self.ind_var, self.b)]
        rss = np.sum(self.y*self.y)-mse_tmp
        d = np.sum(self.ind_var,axis=1)
        if self.isCp:
            self.Cp = rss + 2 * d * rss[-1]/(self.n - self.p -1)
        if self.isAIC:
            self.AIC = self.n * np.log(rss) + 2 * d

    def cvreg(self):
        k = 10
        indexs = np.array_split(np.random.permutation(np.arange(0, self.n)), k)
        def cvk(ind, index):
            txx = self.xtx[ind][:,ind] - np.dot((self.x[index][:,ind]).T, self.x[index][:,ind])
            txy = self.xty[ind] - np.dot((self.x[index][:,ind]).T, self.y[index])
            tcoe = solve_sym(txx, txy)
            return np.sum((self.y[index]-np.dot(self.x[index][:,ind],tcoe))**2)    
        self.cverr = np.sum(np.array([[cvk(ind, index) for index in indexs] for ind in self.ind_var]), axis=1)/self.n
     
    def result(self):
        self.Cp_AIC()
        self.cvreg()  
        output_dict = {}

        if self.isCp:
            min_id_Cp = np.argmin(self.Cp)
            name_Cp = names[0:p][self.ind_var[min_id_Cp][1:]]
            Variable_Cp = np.append("截距",name_Cp)
            Coefficient_Cp = self.b[min_id_Cp]
            output_dict["Variable_Cp"] = Variable_Cp
            output_dict["Coefficient_Cp"] = Coefficient_Cp
        if self.isAIC:
            min_id_AIC = np.argmin(self.AIC)
            name_AIC = names[0:p][self.ind_var[min_id_AIC][1:]]
            Variable_AIC = np.append("截距",name_AIC)
            Coefficient_AIC = self.b[min_id_AIC]
            output_dict["Variable_AIC"] = Variable_AIC
            output_dict["Coefficient_AIC"] = Coefficient_AIC

        if self.isCV:
            min_id_CV = np.argmin(self.cverr)
            name_CV = names[0:p][self.ind_var[min_id_CV][1:]]
            Variable_CV = np.append("截距",name_CV)
            Coefficient_CV = self.b[min_id_CV]
            output_dict["Variable_CV"] = Variable_CV
            output_dict["Coefficient_CV"] = Coefficient_CV

        return(output_dict)
    
    
os.chdir("D:/大三下学习/numpy/Best Subset Regression")
sys.path.append("./prostate")
x = np.loadtxt("./prostate/x.txt", delimiter=",")
y = np.loadtxt("./prostate/y.txt", delimiter=",")
names = np.loadtxt("./prostate/names.txt", delimiter=",", dtype=str)

Best = BestSubsetReg(x=x, y=y, inter=1, isCp=1, isAIC=1, isCV=1)
Best.result()

{'Coefficient_AIC': array([ 0.49472926,  0.54399786,  0.5882127 , -0.01644485,  0.10122333,
         0.71490398]),
 'Coefficient_CV': array([ 0.49472926,  0.54399786,  0.5882127 , -0.01644485,  0.10122333,
         0.71490398]),
 'Coefficient_Cp': array([ 0.49472926,  0.54399786,  0.5882127 , -0.01644485,  0.10122333,
         0.71490398]),
 'Variable_AIC': array(['截距', 'lcavol', 'lweight', 'age', 'lbph', 'svi'],
       dtype='<U7'),
 'Variable_CV': array(['截距', 'lcavol', 'lweight', 'age', 'lbph', 'svi'],
       dtype='<U7'),
 'Variable_Cp': array(['截距', 'lcavol', 'lweight', 'age', 'lbph', 'svi'],
       dtype='<U7')}

In [332]:
#笔记注释版(拆开class)

import numpy as np
from scipy import linalg
import os
import sys

#求β系数
def solve_sym(xwx, s):
    L = linalg.cholesky(xwx)
    return linalg.lapack.dpotrs(L, s)[0]

#生成全部子集合，例turnbits_rec(4)
def turnbits_rec(p):
    if(p==1):
        return np.array([[True, False],[True, True]])
    else:
        tmp1 = np.c_[ turnbits_rec(p-1), np.array([False]*(2**(p-1))).reshape((2**(p-1),1))]
        tmp2 = np.c_[ turnbits_rec(p-1), np.array([True]*(2**(p-1))).reshape((2**(p-1),1))]
        return np.r_[tmp1, tmp2]


os.chdir("D:/大三下学习/numpy/Best Subset Regression")
sys.path.append("./prostate")
x = np.loadtxt("./prostate/x.txt", delimiter=",")
y = np.loadtxt("./prostate/y.txt", delimiter=",")
names = np.loadtxt("./prostate/names.txt", delimiter=",", dtype=str)

n, p = x.shape
#如果含有截距项
x = np.c_[np.ones((n,1)),x]
#如果不含有截距项
#x = x
xtx = np.dot(x.T,x)
xty = np.dot(x.T,y)
ind_var = turnbits_rec(p)
#算回归系数β
b = [solve_sym(xtx[ind][:,ind], xty[ind]) for ind in ind_var]

#Cp,AIC
#Cp:
#算残差RSS的简便方法RSS=Y'Y-β'X'Xβ：
#残差≠误差，模型里的叫误差：Y=Xβ+ε；
#残差是Y-Xβ（Y和拟合值的差是残差，所以残差与拟合值Xβ一定是正交的）
#算残差平方和，公式见Best Subset Regression-py.ipynb的评价模型部分：
#(Y-Xβ)'(Y-Xβ)=Y'Y-2Y'Xβ+β'X'Xβ
#公式左边为残差RSS，上式是在测试集里算的，如果在训练集里，后两项是相等的：Y'Xβ与β'X'Xβ相等
#(因为Y=Xβ+残差，而残差是一定与拟合值正交的)
#所以上式放在训练集里就变成了残差RSS=Y'Y-β'X'Xβ，即下式rss计算式
#使用for带入所有β(2^p个)算β'X'Xβ：
mse_tmp = [np.sum(np.dot(xtx[ind][:,ind],b_)*b_)for ind, b_ in zip(ind_var, b)]
rss = np.sum(y*y)-mse_tmp
#有d个变量，按x轴方向求和(原代码是＝0，应该=1)
d = np.sum(ind_var,axis=1)
#公式见模型评价.ipynb的Cp部分Cp =err+2dσ^2/n
#公式的err就是RSS,σ^2的无偏估计是用全模型估计的
#RSS的最后一个数就是它的全模型，rss[-1]/(self.n-self.p)就是σ^2的无偏估计
Cp = rss + 2 * d * rss[-1]/(n-p-1)
AIC = n * np.log(rss) + 2 * d

#CV
##不建议对矩阵分块（直接对X和y分块），因为消耗内存
#一般做index序列：
#permutation做0~n-1的数做重新排列，np.array_split然后划分成K块，其中n-1必须是k的倍数，得到一个列表
#为何打乱顺序：因为连着发生的两个数据可能排过序，如来自一个地区等，必须对样本重新排列
k = 10
indexs = np.array_split(np.random.permutation(np.arange(0, n)), k)
#ind：回归中哪个变量在模型里p+1维的向量，元素是True or False，对应变量是否在里面
#index:列表里任意一个元素拿进来，表示里面这些元素在建模时是不要的
#txx是删除index里数据(第K折)后，剩余数据的X’X
#txy是剩余数据的X'Y
#tcoe是去掉第K折后的系数
#返回：K折交叉验证误差（公式见模型评价.ipynb）Σ(Y-Xβ)^2
def cvk(ind, index):
    txx = xtx[ind][:,ind] - np.dot((x[index][:,ind]).T, x[index][:,ind])
    txy = xty[ind] - np.dot((x[index][:,ind]).T, y[index])
    tcoe = solve_sym(txx, txy)
    return np.sum((y[index]-np.dot(x[index][:,ind],tcoe))**2)
#列表里做二重循环，令index和ind同时循环，固定模型后每次去掉一折做交叉验证，返回RSS。
#把结果变成array，有ind(2^p)行，列数等于折数(K=10)
#原代码双重循环反了+少了一组[]+axis应＝1
cverr = np.sum(np.array([[cvk(ind, index) for index in indexs] for ind in ind_var]), axis=1)/n

#找到最小值并输出对应的变量和系数
min_id_cv = np.argmin(cverr)
name_cv = names[0:8][ind_var[min_id_cv][1:]]
Variable_CV = np.append("截距",name_cv)
Coefficient_CV = b[min_id_cv]

min_id_Cp = np.argmin(Cp)
name_Cp = names[0:8][ind_var[min_id_Cp][1:]]
Variable_Cp = np.append("截距",name_Cp)
Coefficient_Cp = b[min_id_Cp]

min_id_AIC = np.argmin(AIC)
name_AIC = names[0:8][ind_var[min_id_AIC][1:]]
Variable_AIC = np.append("截距",name_AIC)
Coefficient_AIC = b[min_id_AIC]

dict(zip(['Coefficient_Cp','Variable_Cp','Coefficient_AIC','Variable_AIC','Coefficient_CV','Variable_CV'], 
         [Coefficient_Cp,Variable_Cp,Coefficient_AIC,Variable_AIC,Coefficient_CV,Variable_CV]))


{'Coefficient_AIC': array([ 0.49472926,  0.54399786,  0.5882127 , -0.01644485,  0.10122333,
         0.71490398]),
 'Coefficient_CV': array([-0.77715664,  0.52585188,  0.66176991,  0.66566656]),
 'Coefficient_Cp': array([ 0.49472926,  0.54399786,  0.5882127 , -0.01644485,  0.10122333,
         0.71490398]),
 'Variable_AIC': array(['截距', 'lcavol', 'lweight', 'age', 'lbph', 'svi'],
       dtype='<U7'),
 'Variable_CV': array(['截距', 'lcavol', 'lweight', 'svi'],
       dtype='<U7'),
 'Variable_Cp': array(['截距', 'lcavol', 'lweight', 'age', 'lbph', 'svi'],
       dtype='<U7')}